## 测试Colab上运行Redis的基本功能

- 数据的增、删、改、查

- 数据的持久化

- 基本设置的查看及改写

In [ ]:
import redis_server
!$redis_server.REDIS_SERVER_PATH --daemonize yes

In [ ]:
import redis
r = redis.StrictRedis(host='127.0.0.1', port=6379, db=0)
r.ping()

True

In [ ]:
r.config_set('dir', '/content/redis-rdb/')

True

In [ ]:
r.config_get('dir')

{'dir': '/content'}

In [ ]:
r.config_rewrite()

In [ ]:
r.set('foo', 'bar')

True

In [ ]:
r.get('foo')

b'bar'

In [ ]:
r.set('foo', 'test')
r.set('key1', 'value1')
r.set('key2', 'value2')
r.set('key3', 'value3')
r.set('key4', 'value4')

True

In [ ]:
r.get('key3')

b'value3'

In [ ]:
r.bgsave()

True

In [ ]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)
r.ping()

True

In [ ]:
r.dbsize()

5

In [ ]:
r.ping()

True

In [ ]:
r.save()

True

In [ ]:
r.shutdown(save=True)

In [ ]:
r.get('foo')

In [ ]:
r.config_get()

{'rdbchecksum': 'yes',
 'daemonize': 'yes',
 'io-threads-do-reads': 'no',
 'lua-replicate-commands': 'yes',
 'always-show-logo': 'no',
 'protected-mode': 'yes',
 'rdbcompression': 'yes',
 'rdb-del-sync-files': 'no',
 'activerehashing': 'yes',
 'stop-writes-on-bgsave-error': 'yes',
 'dynamic-hz': 'yes',
 'lazyfree-lazy-eviction': 'no',
 'lazyfree-lazy-expire': 'no',
 'lazyfree-lazy-server-del': 'no',
 'lazyfree-lazy-user-del': 'no',
 'repl-disable-tcp-nodelay': 'no',
 'repl-diskless-sync': 'no',
 'gopher-enabled': 'no',
 'aof-rewrite-incremental-fsync': 'yes',
 'no-appendfsync-on-rewrite': 'no',
 'cluster-require-full-coverage': 'yes',
 'rdb-save-incremental-fsync': 'yes',
 'aof-load-truncated': 'yes',
 'aof-use-rdb-preamble': 'yes',
 'cluster-replica-no-failover': 'no',
 'cluster-slave-no-failover': 'no',
 'replica-lazy-flush': 'no',
 'slave-lazy-flush': 'no',
 'replica-serve-stale-data': 'yes',
 'slave-serve-stale-data': 'yes',
 'replica-read-only': 'yes',
 'slave-read-only': 'yes',
 

## 梳理数据与Redis交互的逻辑


In [ ]:
def check_key(key, value):
    if r.exists(key):
        print('The key is already exists: ', r.get(key))
    else:
        r.set(key, value)
        print('Add the new key-value pair: ', key, value)

check_key('key5', 'value5')

Add the new key-value pair:  key5 value5


## 设计数据保存的结构并进行更新验证

In [ ]:
!pip install redis-server redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.7 MB/s 
     |████████████████████████████████| 246 kB 52.0 MB/s 


In [ ]:
import redis_server
!$redis_server.REDIS_SERVER_PATH --daemonize yes

import redis
redis_codegen_350M = redis.StrictRedis(host='127.0.0.1', port=6379, db=0)
redis_codegen_6B = redis.StrictRedis(host='127.0.0.1', port=6379, db=1)

#### 对数值进行原子化操作

- hset -- mapping 方法下的数据结构满足目前的使用需求

- 后续需对不同模型推理的数据进行分库存储（待验证）

In [ ]:
r.delete('mylist')

1

In [ ]:
redis_codegen_350M.rpush('mylist', 1, 2, 3, 4)

4

In [ ]:
r.rpush('mylist', -20, 40)

6

In [ ]:
r.sort('mylist')

[b'-20', b'1', b'2', b'3', b'4', b'40']

In [ ]:
r.hset("import numpy", mapping={"codegen-350M":"import numpy as np", "num_requests": 1, "adopted":0})

3

In [ ]:
r.hkeys('import numpy')

[b'codegen-350M', b'num_requests', b'adopted']

In [ ]:
r.hget('import numpy', 'num_requests')

b'1'

In [ ]:
r.hset('import numpy', "num_requests", 2)

0

In [ ]:
r.hget('import numpy', 'num_requests')

b'2'

In [ ]:
old_num = r.hget('import numpy', 'num_requests')
new_num = int(old_num) + 10
r.hset('import numpy', 'num_requests', new_num)
r.hget('import numpy', 'num_requests')

b'12'

### 验证多数据的分别存储及查询

- redis_codegen_350M

- redis_codegen_6B

In [ ]:
redis_codegen_350M.hset("import numpy", mapping={"codegen-350M":"import numpy as np", "num_requests": 1, "adopted":0})
redis_codegen_6B.hset("import numpy", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 5, "adopted":3})

3

In [ ]:
redis_codegen_350M.hget('import numpy', 'codegen-350M')

b'import numpy as np'

In [ ]:
redis_codegen_350M.dbsize()

1

In [ ]:
redis_codegen_6B.hget('import numpy', 'codegen-6B')

b'import numpy as np \nimport pandas as pd'

In [ ]:
redis_codegen_6B.hset("import numpy as ", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 6, "adopted":2})
redis_codegen_6B.hset("import numpy as numpy", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 501, "adopted":89})
redis_codegen_6B.hset("import numpy as numpy \nimport pandas", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 17, "adopted":2})


redis_keys = redis_codegen_6B.keys()
print('redis_6B_keys: ', redis_keys)

for key in redis_keys:
    value = redis_codegen_6B.hgetall(key)
    print(value)

redis_6B_keys:  [b'import numpy as numpy', b'import numpy', b'import numpy as ', b'import numpy as numpy \nimport pandas']
{b'codegen-6B': b'import numpy as np \nimport pandas as pd', b'num_requests': b'501', b'adopted': b'89'}
{b'codegen-6B': b'import numpy as np \nimport pandas as pd', b'num_requests': b'5', b'adopted': b'3'}
{b'codegen-6B': b'import numpy as np \nimport pandas as pd', b'num_requests': b'6', b'adopted': b'2'}
{b'codegen-6B': b'import numpy as np \nimport pandas as pd', b'num_requests': b'17', b'adopted': b'2'}


In [ ]:
for s in redis_codegen_6B.scan_iter():
    print(s)

b'import numpy as numpy'
b'import numpy'
b'import numpy as '
b'import numpy as numpy \nimport pandas'


In [ ]:
redis_codegen_6B.mget('import numpy as numpy')

[None]

### 不同方法的速度测试

In [ ]:
import redis
import time
redis_cache = redis.Redis(host='my_redis_host)

time_start = time.time()
for i in range(100):
    for key in redis_cache.scan_iter(match='MONITORABLE-*', count=100):
        pass
print time.time() - time_start

time_start = time.time()
for i in range(100):
    for key in redis_cache.keys('MONITORABLE-*'):
        pass
print time.time() - time_start

### 验证键值数据使用时间戳进行筛选的功能

- 目标：使用数据的创建时间对 key-value数据进行区分


In [ ]:
import time
from datetime import datetime

r = redis.Redis()
r.set('dt', time.time())
r.set_response_callback('GET', float)
result = datetime.fromtimestamp(r.get('dt'))

### 验证数据结构的有效性(已验证)

- 结论：该设计的数据结构不可用

In [ ]:
r.hset("import", mapping={"codegen-350M":'{"codehint":"import numpy as np", "num_requests": 1, "adopted":0}', "codegen-6B": '{"codehint":"import numpy as np \nimport pandas as np", "num_requests": 3, "adopted":5}'})

2

In [ ]:
codegen_6B_dict = r.hget('import', 'codegen-6B')
print('codegen_6B_dict: ', codegen_6B_dict)

codegen_6B_dict:  b'{"codehint":"import numpy as np \nimport pandas as np", "num_requests": 3, "adopted":5}'


In [ ]:
import json
import ast
d = ast.literal_eval(str(codegen_6B_dict))
# d = json.loads(str(codegen_6B_dict))
print(d)

b'{"codehint":"import numpy as np \nimport pandas as np", "num_requests": 3, "adopted":5}'


In [ ]:
r.hmset("import num", {"codegen-350M": ["import numpy as np", "1", "0"], "codegen-6B": ["import numpy as np import pandas as np",  "3", "5"]})

In [ ]:
con.hmset("pythonDict", {"Location": "Ahmedabad", "Company": ["A/C Pri...: sm", "ECW", "Musikaar"]})

In [ ]:
{"codehint":"import numpy as np", "num_requests": 1, "adopted":0}

{'codehint': 'import numpy as np', 'num_requests': 1, 'adopted': 0}

In [ ]:
import json
h = r.hget('import num', 'codegen-6B')
h = h.decode("utf-8") 
d = json.loads(h)
print('d: ', d['adopted'])

d:  5


# Redis数据库遍历及切分方法验证（2022-11-09）

## 1、验证基于 timestamp --时间戳的Redis数据库切分 (取key)

- 01--验证使用scan_iter全部keys值及速度

- 02--验证使用timestamp进行切分的有效性

## 2、模型推理

- 01--多线程推理

- 02--数据存储


## 3、后端代码重构

- 01--数据发送格式重构，json对数据进行打包

- 02--信息收集的相关代码调整

In [1]:
!pip install redis-server redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.6 MB/s 
     |████████████████████████████████| 246 kB 35.8 MB/s 


In [2]:
import redis_server
!$redis_server.REDIS_SERVER_PATH --daemonize yes

import redis
redis_codegen_350m = redis.StrictRedis(host='127.0.0.1', port=6379, db=0, decode_responses=True)
redis_codegen_6b = redis.StrictRedis(host='127.0.0.1', port=6379, db=1, decode_responses=True)

In [3]:
redis_codegen_350m.hset("import n", mapping={"codegen-350M":"import numpy as np", "num_requests": 0, "adopted":0,    "time":1667985359})
redis_codegen_350m.hset("import nu", mapping={"codegen-350M":"import numpy as np", "num_requests": 0, "adopted":0,   "time":1667985124})
redis_codegen_350m.hset("import num", mapping={"codegen-350M":"import numpy as np", "num_requests": 0, "adopted":0,  "time":1667985859})
redis_codegen_350m.hset("import nump", mapping={"codegen-350M":"import numpy as np", "num_requests": 0, "adopted":0, "time":1667985365})

4

In [ ]:
redis_codegen_6b.hset("import n", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 0, "adopted":0,    "time":1667985478})
redis_codegen_6b.hset("import nu", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 0, "adopted":0,   "time":1667985369})
redis_codegen_6b.hset("import num", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 0, "adopted":0,  "time":1667985842})
redis_codegen_6b.hset("import nump", mapping={"codegen-6B":"import numpy as np \nimport pandas as pd", "num_requests": 0, "adopted":0, "time":1667985359})

1

In [ ]:
redis_codegen_350m.hget("import n", "codegen-350M")

b'import numpy as np'

In [ ]:
redis_codegen_350m.dbsize()

4

In [4]:
redis_codegen_350m.hgetall("import n")

{'codegen-350M': 'import numpy as np',
 'num_requests': '0',
 'adopted': '0',
 'time': '1667985359'}

In [ ]:
redis_codegen_350m.hgetall('import n')

{b'codegen-350M': b'import numpy as np',
 b'num_requests': b'0',
 b'adopted': b'0',
 b'time': b'1667985359'}

In [ ]:
from datetime import datetime

current_time = datetime.now()
print('Current time: ', current_time)

Current time:  2022-11-09 09:16:25.655506


In [ ]:
from datetime import datetime
curr_dt = datetime.now()
 
print("Current datetime: ", curr_dt)
timestamp = int(round(curr_dt.timestamp()))
 
print("Integer timestamp of current datetime: ", timestamp)

Current datetime:  2022-11-09 09:15:58.661540
Integer timestamp of current datetime:  1667985359


In [ ]:
import time

time_1 = time.time()
print('time_1: ', time_1)

time_now = datetime.fromtimestamp(time_1)
print("Time_now: ", time_now)

time_1:  1667985769.6175807
Time_now:  2022-11-09 09:22:49.617581


### How to get string type in response rather than bytestring type?

- [web-link](https://github.com/redis/redis-py/issues/910)

In [ ]:
import redis

r = redis.Redis(decode_responses=True)

r.set('foo', 'bar')
r.get('foo')
type(r.get('foo'))

### 1.1、验证使用scan_iter全部keys值及速度

In [6]:
for key in redis_codegen_350m.scan_iter():
    print(key)

import num
import n
import nump
import nu


In [7]:
if redis_codegen_350m.exists('import nu'):
    print('hello')

hello


In [12]:
# 验证使用timestamp对数据进行分类筛选，然后判断收集key值

for key in redis_codegen_350m.scan_iter():
    timestamp = redis_codegen_350m.hget(key, "time")
    print(timestamp)
    if int(timestamp) > int(100):
        print("The key which time is large than 100: ", key)

1667985859
The key which time is large than 100:  import num
1667985359
The key which time is large than 100:  import n
1667985365
The key which time is large than 100:  import nump
1667985124
The key which time is large than 100:  import nu


In [15]:
from datetime import datetime

codehint_db = ["import time", "import numpy", "from matplotlib", "plt.plot", "for i in range", "with open()", "if alpha > "]

for codehint in codehint_db:
    cur_dt = datetime.now()
    timestamp = int(round(cur_dt.timestamp()))
    redis_codegen_350m.hset(codehint, mapping={"codegen_350m": codehint, "num_requests": 0, "time": timestamp})

for key in redis_codegen_350m.scan_iter():
    print(redis_codegen_350m.hgetall(key))
    print("****\n")

{'codegen-350M': 'import numpy as np', 'num_requests': '0', 'adopted': '0', 'time': '1667985859'}
****

{'codegen_350m': 'import time', 'num_requests': '0', 'time': '1668063370'}
****

{'codegen_350m': 'if alpha > ', 'num_requests': '0', 'time': '1668063370'}
****

{'codegen-350M': 'import numpy as np', 'num_requests': '0', 'adopted': '0', 'time': '1667985359'}
****

{'codegen_350m': 'import numpy', 'num_requests': '0', 'time': '1668063370'}
****

{'codegen_350m': 'with open()', 'num_requests': '0', 'time': '1668063370'}
****

{'codegen-350M': 'import numpy as np', 'num_requests': '0', 'adopted': '0', 'time': '1667985124'}
****

{'codegen_350m': 'plt.plot', 'num_requests': '0', 'time': '1668063370'}
****

{'codegen-350M': 'import numpy as np', 'num_requests': '0', 'adopted': '0', 'time': '1667985365'}
****

{'codegen_350m': 'from matplotlib', 'num_requests': '0', 'time': '1668063370'}
****

{'codegen_350m': 'for i in range', 'num_requests': '0', 'time': '1668063370'}
****

